In [1]:
import os

In [2]:
os.chdir(r"E:\Course\Deep Learning Computer Vision™ CNN, OpenCV, YOLO, SSD & GANs\12. Types of Optimizers, Learning Rates & Callbacks Build a Fruit Classifier\fruits-360")

In [3]:
os.getcwd()

'E:\\Course\\Deep Learning Computer Vision™ CNN, OpenCV, YOLO, SSD & GANs\\12. Types of Optimizers, Learning Rates & Callbacks Build a Fruit Classifier\\fruits-360'

In [4]:
std_img_dim  = (32,32,3)

In [5]:
train = os.getcwd()+'\\'+os.listdir()[-2]
validation = os.getcwd()+'\\'+os.listdir()[-1]

In [6]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Flatten, Dense, Convolution2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
img_gen = ImageDataGenerator(rotation_range = 20, width_shift_range = 0.1, height_shift_range = 0.1, shear_range = 0.1, horizontal_flip = True, rescale = 1./255)

In [9]:
train_img_gen = img_gen.flow_from_directory(train, target_size = std_img_dim[:2], batch_size = 10)
validation_img_gen = img_gen.flow_from_directory(validation, target_size = std_img_dim[:2], shuffle = False, batch_size = 10)

Found 13877 images belonging to 81 classes.
Found 0 images belonging to 0 classes.


In [14]:
model = Sequential()

In [15]:
model.add(Convolution2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = std_img_dim))
model.add(Convolution2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = std_img_dim))

In [16]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [17]:
model.add(Dropout(0.25))

In [18]:
model.add(Convolution2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = std_img_dim))
model.add(Convolution2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = std_img_dim))

In [19]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [20]:
model.add(Dropout(0.25))

In [21]:
model.add(Flatten())

In [22]:
model.add(Dense(512, activation = 'relu'))

In [23]:
model.add(Dense(81, activation = 'softmax'))

In [24]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = 'accuracy')

In [26]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [27]:
early_stop = EarlyStopping(monitor = 'accuracy', patience = 2)
reduce_lr = ReduceLROnPlateau(monitor = 'accuracy', factor = 0.2, patience = 2)
model_checkp = ModelCheckpoint(filepath = os.getcwd(), monitor = 'accuracy', save_best_only = True)

In [28]:
model.fit(train_img_gen, batch_size = 32, epochs = 10, validation_data = validation_img_gen, callbacks = [early_stop, reduce_lr, model_checkp])

Epoch 1/10
4133/4133 [==============================] - ETA: 0s - loss: 0.9150 - accuracy: 0.7282WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: E:\Course\Deep Learning Computer Vision™ CNN, OpenCV, YOLO, SSD & GANs\12. Types of Optimizers, Learning Rates & Callbacks Build a Fruit Classifier\fruits-360\assets
4133/4133 [==============================] - 2339s 566ms/step - loss: 0.9150 - accuracy: 0.7282 - val_loss: 0.5696 - val_accuracy: 0.8706
Epoch 2/10
4133/4133 [==============================] - 254s 61ms/step - loss: 0.2587 - 

In [29]:
from sklearn.metrics import classification_report
import numpy as np

In [30]:
prediction = np.argmax(model.predict(validation_img_gen), axis=-1)


In [31]:
prediction

array([ 0,  0,  0, ..., 80, 80, 80], dtype=int64)

In [32]:
print(classification_report(validation_img_gen.classes, prediction))

              precision    recall  f1-score   support

           0       0.80      0.87      0.83       164
           1       0.97      0.99      0.98       164
           2       0.99      1.00      1.00       164
           3       0.78      0.94      0.85       161
           4       1.00      0.74      0.85       164
           5       0.81      0.89      0.85       164
           6       1.00      0.77      0.87       164
           7       0.93      0.92      0.92       144
           8       1.00      1.00      1.00       166
           9       0.84      0.84      0.84       164
          10       1.00      0.98      0.99       164
          11       1.00      0.98      0.99       143
          12       1.00      1.00      1.00       166
          13       0.96      0.96      0.96       166
          14       0.98      0.99      0.99       166
          15       0.96      0.96      0.96       166
          16       1.00      1.00      1.00       164
          17       0.96    

In [64]:
os.chdir(r"D:\B2B_Git_Instance\CNN_Keras")

model.save("fruit_classifier.h5")

In [16]:
from kerastuner import HyperModel, Hyperband

In [57]:
from kerastuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        model.add(
            Convolution2D(
                filters=hp.Int(name = 'conv_1', min_value = 32, max_value = 64, step = 16),
                kernel_size=hp.Choice(name= 'conv_1', values  = [3,6]),
                activation='relu',
                input_shape=self.input_shape, padding = 'same'
            )
        )
        model.add(
            Convolution2D(
                filters=hp.Int(name = 'conv_2', min_value = 32, max_value = 64, step = 16),
                kernel_size=hp.Choice(name= 'conv_2', values  = [3,6]),
                activation='relu',
                input_shape=self.input_shape, padding = 'same'
            )
        )
        model.add(MaxPooling2D(pool_size=hp.Choice(name = 'max_pool_1', values = [2,4])))
        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05
            ))
        )
        model.add(
            Convolution2D(
                filters=hp.Int(name = 'conv_3', min_value = 64, max_value = 128, step = 16),
                kernel_size=hp.Choice(name= 'conv_3', values  = [3,6]),
                activation='relu',
                input_shape=self.input_shape, padding = 'same'
            )
        )
        model.add(
            Convolution2D(
                filters=hp.Int(name = 'conv_4', min_value = 64, max_value = 128, step = 16),
                kernel_size=hp.Choice(name= 'conv_4', values  = [3,6]),
                activation='relu',
                input_shape=self.input_shape, padding = 'same'
            )
        )
        model.add(MaxPooling2D(pool_size=hp.Choice(name = 'max_pool_2', values = [2,4])))
        model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        model.add(Flatten())
        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=512,
                    max_value=512**2,
                    step=32,
                    default=512
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_3',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        model.add(Dense(self.num_classes, activation='softmax'))
        model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = 'accuracy')
        return model

hypermodel = CNNHyperModel(input_shape=std_img_dim, num_classes=81)


In [58]:
tuner = Hyperband(
    hypermodel,
    max_epochs=5,
    objective='val_accuracy',
    seed=1429,
    executions_per_trial=5,
    directory='hyperband',
    project_name='fruitClassifier'
)

In [59]:
tuner.search_space_summary()

tuner.search(train_img_gen, validation_img_gen, epochs=10)


In [ ]:
import os

In [83]:
os.getcwd()

'E:\\Course\\Deep Learning Computer Vision™ CNN, OpenCV, YOLO, SSD & GANs\\12. Types of Optimizers, Learning Rates & Callbacks Build a Fruit Classifier\\fruits-360'

In [84]:
os.chdir(r"D:\B2B_Git_Instance\CNN_Keras")

In [85]:
model.save("fruit_classifier.h5")